In [1]:
from tbparse import SummaryReader
import pandas as pd
import os

In [2]:
def fix_display():
    display = pd.options.display
    display.max_columns = 1000
    display.max_rows = 1000
    display.max_colwidth = 199
    display.width = 1000


def get_eval(reader):
    tensors = reader.tensors
    filtered_steps = tensors[~tensors['tag'].str.contains('eval')]
    filtered_steps = filtered_steps[filtered_steps['step'] == tensors['step'].max()]
    eval = filtered_steps[['tag', 'value']]
    return eval


def get_train_time(reader):
    tensors = reader.tensors
    steps_max = tensors['step'].max()
    steps_sum = tensors[tensors['tag'] == 'steps_per_sec'].value.sum()
    step_count = tensors[tensors['tag'] == 'steps_per_sec'].value.count()
    mean_step_per_sec = steps_sum / step_count
    train_time_minutes = steps_max / mean_step_per_sec / 60
    return round(train_time_minutes, 2)


In [3]:
# models = ['my_ssd_mobnet_20_b8', 'my_ssd_mobnet_20_b16', 'my_ssd_mobnet_20_b64'] #batchsize
# models = ['my_ssd_mobnet_5', 'my_ssd_mobnet_10', 'my_ssd_mobnet_20', 'my_ssd_mobnet_50', 'my_ssd_mobnet_100', 'my_ssd_mobnet_200'] #steps
# models = [
#     'ssd_mobilenet_v2_fpnlite_640px_b1_50',
#     'ssd_mobilenet_v2_fpnlite_640px_b2_50',
#     'ssd_mobilenet_v2_fpnlite_640px_b4_50',
#     'ssd_mobilenet_v2_fpnlite_640px_b4_100'
# ]  #batch

# models = [
#     'my_ssd_mobnet_200_b1',
#     'my_ssd_mobnet_100_b2',
#     'my_ssd_mobnet_50',
#     'my_ssd_mobnet_25_b8',
#     'my_ssd_mobnet_12_b16',
#     'my_ssd_mobnet_6_b32',
# ]
#
#BEST
models = [
    'centernet_hg104_512x512_coco17_tpu-8_100_b2',
    'centernet_mobilenetv2fpn_512x512_coco17_od_test_lr2',
    'efficentdet_d0_512_50_b8',
    'efficentdet_d1_640_b4_50',
    'ssd_mobnet_v2_zwykly_320px_b4_50_sett1',
    'faster_rcnn_resnet50_v1_640x640_150',
    'my_ssd_mobnet_100',
    'ssd_mobilenet_v2_fpnlite_640px_b4_100',
    'ssd_mobnet_v2_zwykly_320px_b8_100_lr1',
    'ssd_resnet50_v1_fpn_640_b2_100_l2'
]

In [4]:
result = None
for i, model in enumerate(models):
    eval_dir = os.path.join(f'Tensorflow/workspace/models/{model}/eval')
    train_dir = os.path.join(f'Tensorflow/workspace/models/{model}/train')
    reader_eval = SummaryReader(eval_dir)
    eval = get_eval(reader_eval)
    eval.rename(columns={'value': model}, inplace=True)
    reader_train = SummaryReader(train_dir)
    train_time = get_train_time(reader_train)
    eval = pd.concat([eval, pd.DataFrame([{'tag': 'Train time [m]', model: train_time}])])
    if result is None:
        result = eval.copy(deep=True)
        continue
    result = pd.merge(result, eval, how='inner', on=['tag'])

result

,tag,centernet_hg104_512x512_coco17_tpu-8_100_b2,centernet_mobilenetv2fpn_512x512_coco17_od_test_lr2,efficentdet_d0_512_50_b8,efficentdet_d1_640_b4_50,ssd_mobnet_v2_zwykly_320px_b4_50_sett1,faster_rcnn_resnet50_v1_640x640_150,my_ssd_mobnet_100,ssd_mobilenet_v2_fpnlite_640px_b4_100,ssd_mobnet_v2_zwykly_320px_b8_100_lr1,ssd_resnet50_v1_fpn_640_b2_100_l2
0,DetectionBoxes_Precision/mAP,0.233697,0.30013,0.250799,0.270638,0.030932,0.116373,0.218108,0.245905,0.233799,0.207044
1,DetectionBoxes_Precision/mAP (large),-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,DetectionBoxes_Precision/mAP (medium),0.333695,0.348219,0.442434,0.411033,0.106253,0.203612,0.363053,0.363027,0.405577,0.386758
3,DetectionBoxes_Precision/mAP (small),0.224927,0.297703,0.234082,0.256016,0.027537,0.115321,0.210458,0.240066,0.224372,0.193083
4,DetectionBoxes_Precision/mAP@.50IOU,0.493124,0.623875,0.593896,0.599025,0.13661,0.314888,0.499325,0.508182,0.563689,0.40558
5,DetectionBoxes_Precision/mAP@.75IOU,0.186808,0.242577,0.148875,0.199604,0.001926,0.053038,0.141765,0.198651,0.1513,0.19243
6,DetectionBoxes_Recall/AR@1,0.397876,0.422315,0.376367,0.40347,0.117013,0.29679,0.353116,0.370654,0.370206,0.385196
7,DetectionBoxes_Recall/AR@10,0.498102,0.517024,0.441333,0.493054,0.165483,0.364605,0.450525,0.488884,0.439945,0.4938
8,DetectionBoxes_Recall/AR@100,0.498102,0.517024,0.453507,0.506077,0.170289,0.40783,0.473827,0.501785,0.446018,0.522111
9,DetectionBoxes_Recall/AR@100 (large),-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [5]:
result.to_excel('table.xlsx', index=False)
!table.xlsx